<a href="https://colab.research.google.com/github/ML-Bioinfo-CEITEC/genomic_benchmarks/blob/main/notebooks/How_To_Train_CNN_Classifier_With_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How To Train CNN Classifier With TF

This notebook demonstrates how to use `genomic_benchmarks` to train a neural network classifier on one of its benchmark datasets [human_nontata_promoters](https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks/tree/main/docs/human_nontata_promoters).

In [2]:
pip install genomic-benchmarks

  Cloning https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks.git to /tmp/pip-req-build-ib04l8_s
  Running command git clone -q https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks.git /tmp/pip-req-build-ib04l8_s
     |████████████████████████████████| 2.3 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 40.5 MB/s 
     |████████████████████████████████| 271 kB 52.3 MB/s 
     |████████████████████████████████| 160 kB 47.7 MB/s 
  Created wheel for genomic-benchmarks: filename=genomic_benchmarks-0.0.3-py3-none-any.whl size=25944 sha256=411bd9a709acbb5ed831116bc1a83e92ad16e4b014cd0b6d9a5759774af2fb96
  Stored in directory: /tmp/pip-ephem-wheel-cache-5itcmaud/wheels/b2/22/e4/91b7bd669628972e32032ea3c61a4bf5d0d33bb752fe8674d5
Successfully built genomic-benchmarks
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Data download

With the function `download_dataset` downloads, we can download full-sequence form of the benchmark, splitted into train and test sets, one folder for each class.

In [3]:
from pathlib import Path
import tensorflow as tf

from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model

if not is_downloaded('human_nontata_promoters'):
    download_dataset('human_nontata_promoters')

/usr/local/lib/python3.7/dist-packages/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


/usr/local/lib/python3.7/dist-packages/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Done.
Unzipping...Done.


In [4]:
info('human_nontata_promoters', 0)

Dataset `human_nontata_promoters` has 2 classes: negative, positive.

All lenghts of genomic intervals equals 251.

Totally 36131 sequences have been found, 27097 for training and 9034 for testing.


,train,test
negative,12355,4119
positive,14742,4915


## TF Dataset object

To train the model with TensorFlow, we must create a TF Dataset. Because the directory structure of our benchmarks is ready for training, we can just call `tf.keras.preprocessing.text_dataset_from_directory` function as follows.

In [5]:
BATCH_SIZE = 64
SEQ_PATH = Path.home() / '.genomic_benchmarks' / 'human_nontata_promoters'
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 27097 files belonging to 2 classes.


## Text vectorization

To convert the strings to tensors, we internally use TF `TextVectorization` layer and splitting to characters.

In [6]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
#vectorize_layer.set_vocabulary(vocab=np.asarray(['a', 'c', 't', 'g', 'n']))
vectorize_layer.get_vocabulary()

['', '[UNK]', 'g', 'c', 't', 'a']

In [7]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

To get a baseline (other models can be compared to) we ship a package with [a simple CNN model](https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks/blob/main/src/genomic_benchmarks/models/tf.py). We have vectorized the dataset before training the model to speed up the process.

In [8]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [9]:
EPOCHS = 10

history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10
424/424 [==============================] - 43s 26ms/step - loss: 0.4436 - binary_accuracy: 0.7975
Epoch 2/10
424/424 [==============================] - 11s 26ms/step - loss: 0.3861 - binary_accuracy: 0.8257
Epoch 3/10
424/424 [==============================] - 11s 25ms/step - loss: 0.3606 - binary_accuracy: 0.8403
Epoch 4/10
424/424 [==============================] - 11s 25ms/step - loss: 0.3402 - binary_accuracy: 0.8509
Epoch 5/10
424/424 [==============================] - 11s 26ms/step - loss: 0.3198 - binary_accuracy: 0.8589
Epoch 6/10
424/424 [==============================] - 11s 25ms/step - loss: 0.3058 - binary_accuracy: 0.8653
Epoch 7/10
424/424 [==============================] - 11s 25ms/step - loss: 0.2954 - binary_accuracy: 0.8716
Epoch 8/10
424/424 [==============================] - 11s 25ms/step - loss: 0.2816 - binary_accuracy: 0.8803
Epoch 9/10
424/424 [==============================] - 11s 26ms/step - loss: 0.2699 - binary_accuracy: 0.8847
Epoch 10/10
424/424

## Evaluation on the test set

Finally, we can do the same pre-processing for the test set and evaluate the F1 score of our model.

In [10]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES,
    shuffle=False)

test_ds =  test_dset.map(vectorize_text)

Found 9034 files belonging to 2 classes.


In [11]:
from sklearn.metrics import f1_score
from genomic_benchmarks.data_check.info import labels_in_order

predictions = [int(x[0] > 0) for x in model.predict(test_ds)]
labels = labels_in_order(dset_name='human_nontata_promoters')
f1_score(labels, predictions)


0.8238763713577917